In [1]:
import numpy as np

In [2]:
# convert from polar coordinates to cartesian coordinates
def get_cards_from_pol (x):
  '''
  # Suyash
  This function takes in an n-dimensional vector defined in the cartesian coordinates and returns the 
  vector in its polar coordinates.
  '''
  y = np.zeros(len(x))
  for j in range(len(y)):
    val = 1
    i=0
    while i < len(x) - j-1:
        val = val*np.cos(x[i+1])
        i=i+1
    if j>0:
      val = val*np.sin(x[i+1])
    y[j] = x[0]*val  
  return y

In [3]:
#new implementation
def get_carts_from_polar(x):
  '''
  # Manu
  This function takes in an n-dimensional vector defined in the cartesian coordinates and returns the 
  vector in its polar coordinates.
  '''
  r = x[0]
  thts = x[1:] #thetas
  thts = thts[::-1] #to make things easier

  y = np.zeros(len(x))
  ld = 1 #lower dim
  current = 0

  for idx, th in enumerate(thts):
    
    current = ld * np.cos(th)
    y[idx] = r*current

    ld = ld * np.sin(th)
    
  y[-1] = r*ld
  y = y[::-1] #to get in xyz format

  return y


# Here I am defining a separate function to carry out all the tasks performed earlier.

In [4]:
def get_count_polar(x):
  '''
  This function takes in a set of m vectors and calculates the number of vectors that are located on
  the surface of an n-dimensional sphere.

  The first element of the vector corresponds to the radius. If the radius is greater than the threshold,
  the count is increased by 1.
  '''
  count = 0 
  for x_r in x:
    if x_r[0]>0.99:
      count+=1
  return count

def get_count_cartesian(x):
  '''
  This function takes in a set of m vectors and calculates the number of vectors that are located on
  the surface of an n-dimensional cube.

  Each element of a vector is compared with a threshold value. If any element of the vector exceeds the
  threshold, the count is increased by 1.
  '''
  count = 0
  for x_r in x:
    d = x_r>0.9999      # increase decimal count decrease
    d = d.astype(int)
    sum = np.sum(d)
    if sum>0:
      count+=1
  return count

def get_distance(x,x_new):
  '''
  This function calculates the Euclidean distance of a new point in the n-dimensional space with all the
  m vectors. 

  Returns the mean distance of the point to every other point and the variance between the distances.
  '''
  distance = []
  for x_r in x:
    dist = np.linalg.norm(x_r-x_new)
    distance.append(dist)
  mean = np.mean(distance)
  variance = np.var(distance)
  return mean,variance


def curse(n_dimensions,n_samples):
  '''
  This function performs the following tasks in both cartesian and polar coordinates:

  1. Define m n-dimensional points/vectors.
  2. Calculate the number of points on the surface of an n-dimensional object
  3. Calculate the distance of a random point in the n-dimensional space with all the m-points
     and compute the mean distance and variance in the distances.

  Input arguments:
  n_dimensions : The length of the vector (dimensionality)
  n_samples : Number of vectors in the n-dimensional space

  Output argument:
  count_cart : Number of points on the surface of the n-dimensional cube
  mean_dist_cart : Mean distance of a point from all the points (points defined in cartesian coordinates)
  var_dist_cart : Variance in the distances (points defined in cartesian coordinates)
  count_pol : Number of points on the surface of the n-dimensional sphere
  mean_dist_pol : Mean distance of a point from all the points (points defined in polar coordinates)
  var_dist_pol : Variance in the distances (points defined in polar coordinates)
  '''
  x = np.array([[np.random.uniform() for i in range(n_dimensions)] for j in range(n_samples)]) # define a list of m n-dimensional vectors
  x_new = np.array([np.random.uniform() for i in range(n_dimensions)])  # define a random point in the same space
  
  # cartesian coordinates
  count_cart = get_count_cartesian(x)
  mean_dist_cart, var_dist_cart = get_distance(x,x_new)

  # polar coordinates 
  # vector of the form [r, theta1,theta2,...]
  x[:,1:-1] = x[:,1:-1]*np.pi   # convert the elements to angles.
  x[:,-1]=x[:,-1]*np.pi*2
  x_new[1:] = x_new[1:]*np.pi
  x[-1]=x[-1]*np.pi*2
  count_pol = get_count_polar(x)  #get the count of points on the surface
  

  y = [get_carts_from_polar(x_r) for x_r in x]  # convert to cartesian coordinates
  y_new = get_carts_from_polar(x_new) 

  mean_dist_pol, var_dist_pol = get_distance(y,y_new)
  return count_cart,mean_dist_cart,var_dist_cart,count_pol,mean_dist_pol, var_dist_pol


In [5]:
count_cart,mean_dist_cart, var_dist_cart,count_pol,mean_dist_pol, var_dist_pol=curse(1000,1000)

print("Number of points on surface of n-D cube:",count_cart)
print("Mean distance:", mean_dist_cart)
print("Variance:",var_dist_cart)
print()
print("Number of points on surface of n-D sphere:",count_pol)
print("Mean distance:", mean_dist_pol)
print("Variance:",var_dist_pol)


Number of points on surface of n-D cube: 100
Mean distance: 13.051196364812087
Variance: 0.044008756500694884

Number of points on surface of n-D sphere: 10
Mean distance: 1.0778633659601369
Variance: 0.14210096471024558


In [ ]:
'''
This function is to convert the cartesian coordinates back to polar coordinates. The problem as of
now is that the numpy's arccos returns only angles between 0-pi. However in the original polar coordinates,
the last angle varies between 0-2pi.

This gives a correct answer when all random angles are less than pi but gives error when even one angle
is greater than pi. 

I still have to fix this
'''

def get_pol_from_cart(x):
  '''
  The logic is
  y[0] is the radius - norm of all elements
  y[1] = acos(y/root(x^2+y^2))
  y[2] = acos(z/root(x^2+y^2+z^2))
  .
  .
  .
  
  '''
  x = np.array(x)
  y = np.zeros(len(x))
  y[0] = np.linalg.norm(x)    # radius
  for i in range(1,len(x)):
    y[i] = np.arccos(x[i]/np.linalg.norm(x[:i+1]))
  return y

In [ ]:
x = np.array([np.random.uniform() for i in range(4)])
x_pol = x.copy()
x_pol[1:-1] = x_pol[1:-1]*np.pi
x_pol[-1] = x_pol[-1]*np.pi*2
print("x in polar:",x_pol)
x_cart = get_carts_from_polar(x_pol)
print("x in cartesian:",x_cart)
x_re_pol = get_pol_from_cart(x_cart)
print("x in pol:", x_re_pol)

x in polar: [0.32554158 1.4128717  2.07269645 0.5334497 ]
x in cartesian: [ 0.14331808  0.02282351 -0.07964004  0.28031018]
x in pol: [0.32554158 1.4128717  2.07269645 0.5334497 ]
